In [1]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1920,1080))
display.start()

In [3]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# repeated-dose
genetic_path = '//*[@id="QU.SE.7.5-repeated-dose-toxicity-header"]/div/div[2]'
driver.find_element_by_xpath(genetic_path).click()
time.sleep(0.5)

# header 하위 3개의 창
# dermal
inh_path = '//*[@id="QU.SE.7.5-repeated-dose-toxicity"]/div/div/div[3]/div[3]/button'
driver.find_element_by_xpath(inh_path).click()
time.sleep(0.5)

/tmp/ipykernel_4540/3474652975.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [4]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG422경로
tg422_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[15]'
driver.find_element_by_xpath(tg422_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [5]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [6]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        time.sleep(1)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).send_keys(Keys.ENTER)
            #driver.find_element_by_xpath(accept_path).click()
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg422_dermal_source.json', 'w'))

100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.27s/it, page=6]
79.16314625740051


In [ ]:
##json전처리

In [1]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg422_dermal_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string


#%%
result_ = []

for i in tqdm(range(len(df))):
#for i in tqdm(range(100)):
    CE = np.nan
    EL = np.nan
    RH = np.nan
    CS = np.nan
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn
        
        # Critical effects observed:
        admin_tmp = soup.find_all("dl", {"class" : "HorDL"})

        for j in range(len(admin_tmp)):
            if "<dt>Critical effects observed:" in str(admin_tmp[j]):
                CE = str(admin_tmp[j]).split('<dt>Critical effects observed:</dt><dd>')[1]
                CE = CE.split('</dd>')[0]
            chem_dict['Critical effects'] = CE
            
        # Basis for effect level:
        for j in range(len(admin_tmp)):
            if "<dt>Basis for effect level:" in str(admin_tmp[j]):
                EL = str(admin_tmp[j]).split('<dt>Basis for effect level:</dt><dd>')[1]
                EL = EL.split('</dd>')[0]
            chem_dict['effect level'] = EL
            
        # Relevant for humans:
        for l in range(len(admin_tmp)):
            if "<dt>Relevant for humans:" in str(admin_tmp[l]):
                RH = str(admin_tmp[l]).split('<dt>Relevant for humans:</dt><dd>')[1]
                RH = RH.split('</dd>')[0]
            chem_dict['Relevant_human'] = RH
            
        # Clinical signs:
        for j in range(len(admin_tmp)):
            if "<dt>Clinical signs:" in str(admin_tmp[j]):
                CS = str(admin_tmp[j]).split('<dt>Clinical signs:</dt><dd>')[1]
                CS = CS.split('</dd>')[0]
            chem_dict['Clinical_sign'] = CS
            
        # other results
        result_and_discussion = soup.find('h3', attrs={'id': 'sResultsAndDiscussion'})
        table_list = result_and_discussion.find_next_sibling('div').find_all('dl')
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        #print(f'{i}th error')
        pass
    


# %%
result = pd.DataFrame(result_)

100%|███████████████████████████████████████████████████████████████████████████████████| 26/26 [00:01<00:00, 25.83it/s]


In [2]:
result

Chemical        CasRN  \
0   Extracts (petroleum), deasphalted vacuum resid...   91995-70-9   
1   Extracts (petroleum), deasphalted vacuum resid...   91995-70-9   
2                            Lithium isooctadecanoate   84731-55-5   
3                            Lithium isooctadecanoate   84731-55-5   
4                          5-methylheptan-3-one oxime   22457-23-4   
5                                 Lithium docosanoate    4499-91-6   
6                                 Lithium docosanoate    4499-91-6   
7                                Tallow, lithium salt   68608-50-4   
8                                Tallow, lithium salt   68608-50-4   
9                     Naphthenic acids, lithium salts   61788-56-5   
10                    Naphthenic acids, lithium salts   61788-56-5   
11                 Fatty acids, C16-18, calcium salts   85251-71-4   
12                 Fatty acids, C16-18, calcium salts   85251-71-4   
13                 Fatty acids, C16-18, lithium salts   68783-37-9   
14                 Fatty acids, C16-18, lithium salts   68783-37-9   
15                                 Calcium distearate    1592-23-0   
16                                 Calcium distearate    1592-23-0   
17             Castor oil, hydrogenated, lithium salt   64754-95-6   
18             Castor oil, hydrogenated, lithium salt   64754-95-6   
19  1,2,3,4,4a,7,8,8a-octahydro-2,4a,5,8a-tetramet...   65405-72-3   
20                              Calcium didocosanoate    3578-72-1   
21                              Calcium didocosanoate    3578-72-1   
22                                   Calcium dioleate     142-17-6   
23                                   Calcium dioleate     142-17-6   
24        Fatty acids C18-(unsaturated) lithium salts  502962-81-4   
25        Fatty acids C18-(unsaturated) lithium salts  502962-81-4   
26  Acid Catalysed Reaction Products of 3,7-dimeth...            -   
27  Reaction mass of 3-({[2,2-dimethyl-3-({[2-(2-{...            -   
28                Diallyl 2,2'-oxydiethyl dicarbonate     142-22-3   
29  Overbased reaction products of distilled tall ...            -   
30  Overbased reaction products of distilled tall ...            -   
31       Distillates (petroleum), hydrotreated middle   64742-46-7   
32       Distillates (petroleum), hydrotreated middle   64742-46-7   
33       Distillates (petroleum), hydrotreated middle   64742-46-7   
34       Distillates (petroleum), hydrotreated middle   64742-46-7   
35                         Lithium 12-hydroxystearate    7620-77-1   
36                         Lithium 12-hydroxystearate    7620-77-1   
37            Gas oils (petroleum), hydrodesulfurized   64742-79-6   
38            Gas oils (petroleum), hydrodesulfurized   64742-79-6   
39            Gas oils (petroleum), hydrodesulfurized   64742-79-6   
40            Gas oils (petroleum), hydrodesulfurized   64742-79-6   
41         Extracts (petroleum), residual oil solvent   64742-10-5   
42         Extracts (petroleum), residual oil solvent   64742-10-5   
43  Distillates (petroleum), hydrodesulfurized middle   64742-80-9   
44  Distillates (petroleum), hydrodesulfurized middle   64742-80-9   
45  Distillates (petroleum), hydrodesulfurized middle   64742-80-9   
46  Distillates (petroleum), hydrodesulfurized middle   64742-80-9   

   Critical effects                                       effect level  \
0     not specified  other: overall effects\nHaematology; clinical ...   
1     not specified  other: overall effects\nHaematology; clinical ...   
2     not specified                                other: see 'Remark'   
3     not specified                                other: see 'Remark'   
4                no                                     clinical signs   
5     not specified                                other: see 'Remark'   
6     not specified                                other: see 'Remark'   
7     not specified                                other: see 'Remark'   
8     not specifie

In [3]:

#%%
'''    
    1. critical effect가 우선, not specified는 
    2. effect level의 경우 소문자화한 다음 'no '(띄어쓰기 중요)가 있으면 0 / reduce, decrease, increase등이 있으면 1
    3. 이것도 없다면,,, Clinical signs, Mortality, Body weight and weight changes
    
'''

def find_colon(string):
    return ':' in string


def check_nan(string):
    return string == string
        

# result['Route of administration'].unique()
# result['Route of administration'].value_counts()

# effect 항목의 결과 내용 중 포함이 될 경우
def effective(string):
    if check_nan(string):
        string = string.lower()
        if 'no' in string and 'not' not in string:
            type_ = 0
        
        elif 'yes' in string or 'effects observed, treatment-related' in string:
            type_ = 1
            
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

def conclusion(string):
    if check_nan(string):
        string = string.lower()
        #주요 단어를 적는 곳인듯
        if 'no' in string or 'absence' in string or 'not' in string:
            type_ = 0
        
        elif 'reduce' in string or 'decrease' in string or 'increase' in string or 'reduction' in string or 'adverse' in string or 'gain' in string:
            type_ = 1
        
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

#%%
df = result.copy()

# effect
df['target1'] = df['Critical effects'].map(lambda x: effective(x))
df['target3'] = df['Clinical_sign'].map(lambda x: effective(x))

#effect level
df['target2'] = df['effect level'].map(lambda x: conclusion(x))

# make for result
df = df[['CasRN','target1','Relevant_human','target2','target3']]

In [4]:
# conclude target
df_null = df.isnull()
for i in range(len(df)):
    if df_null['target1'][i] == False:
        target = df['target1'][i]
    elif df_null['target2'][i] == False:
        target = df['target2'][i]
    elif df['target3'][i] == 0:
        target = df['target3'][i]
    else : target = np.nan
    df.loc[i,'target'] = target

In [5]:
df

CasRN  target1  Relevant_human  target2  target3  target
0    91995-70-9      NaN             NaN      NaN        0     0.0
1    91995-70-9      NaN             NaN      NaN        0     0.0
2    84731-55-5      NaN             NaN      NaN        1     NaN
3    84731-55-5      NaN             NaN      NaN        1     NaN
4    22457-23-4      0.0             NaN      NaN        0     0.0
5     4499-91-6      NaN             NaN      NaN        1     NaN
6     4499-91-6      NaN             NaN      NaN        1     NaN
7    68608-50-4      NaN             NaN      NaN        1     NaN
8    68608-50-4      NaN             NaN      NaN        1     NaN
9    61788-56-5      NaN             NaN      NaN        1     NaN
10   61788-56-5      NaN             NaN      NaN        1     NaN
11   85251-71-4      NaN             NaN      NaN        1     NaN
12   85251-71-4      NaN             NaN      NaN        1     NaN
13   68783-37-9      NaN             NaN      NaN        1     NaN
14   68783-37-9      NaN             NaN      NaN        1     NaN
15    1592-23-0      NaN             NaN      NaN        1     NaN
16    1592-23-0      NaN             NaN      NaN        1     NaN
17   64754-95-6      NaN             NaN      NaN        1     NaN
18   64754-95-6      NaN             NaN      NaN        1     NaN
19   65405-72-3      0.0             NaN      NaN        1     0.0
20    3578-72-1      NaN             NaN      NaN        1     NaN
21    3578-72-1      NaN             NaN      NaN        1     NaN
22     142-17-6      NaN             NaN      NaN        1     NaN
23     142-17-6      NaN             NaN      NaN        1     NaN
24  502962-81-4      NaN             NaN      NaN        1     NaN
25  502962-81-4      NaN             NaN      NaN        1     NaN
26            -      0.0             NaN      NaN        1     0.0
27            -      NaN             NaN      0.0        0     0.0
28     142-22-3      NaN             NaN      0.0        0     0.0
29            -      NaN             NaN      NaN        1     NaN
30            -      NaN             NaN      NaN        1     NaN
31   64742-46-7      NaN             NaN      NaN        1     NaN
32   64742-46-7      NaN             NaN      NaN        1     NaN
33   64742-46-7      NaN             NaN      NaN        1     NaN
34   64742-46-7      NaN             NaN      NaN        1     NaN
35    7620-77-1      NaN             NaN      NaN        1     NaN
36    7620-77-1      NaN             NaN      NaN        1     NaN
37   64742-79-6      NaN             NaN      NaN        1     NaN
38   64742-79-6      NaN             NaN      NaN        1     NaN
39   64742-79-6      NaN             NaN      NaN        1     NaN
40   64742-79-6      NaN             NaN      NaN        1     NaN
41   64742-10-5      NaN             NaN      NaN        0     0.0
42   64742-10-5      NaN             NaN      NaN        0     0.0
43   64742-80-9      NaN             NaN      NaN        1     NaN
44   64742-80-9      NaN             NaN      NaN        1     NaN
45   64742-80-9      NaN             NaN      NaN        1     NaN
46   64742-80-9      NaN             NaN      NaN        1     NaN

In [6]:
# save df
df.to_csv('tg422_dermal_0raw.csv', header = True, index = False)

# smiles 추가

In [7]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [8]:
data = pd.read_csv('tg422_dermal_0raw.csv')
len(data['CasRN'].unique())

21

In [9]:
data = data[['CasRN','target']]
data.head()

CasRN  target
0  91995-70-9     0.0
1  91995-70-9     0.0
2  84731-55-5     NaN
3  84731-55-5     NaN
4  22457-23-4     0.0

In [10]:
data['target'].unique()
data['target'].isna().sum()

data = data[data['target'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['target'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [11]:
data = data.drop_duplicates()
len(data)

5

In [12]:
tqdm.pandas()
data['SMILES'] = data.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.34s/it]


In [13]:
data.SMILES.isna().sum()
data = data[data['SMILES'].notna()].reset_index(drop = True)

In [14]:
len(data)

3

In [15]:
data.to_csv('tg422_dermal_1smiles.csv', header=True, index=False)